In [17]:
from dotenv import load_dotenv
import os
import mysql.connector
import pandas as pd
print("DB_HOST:", os.getenv("DB_HOST"))

# Credenciales desde el archivo .env
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

def consultar_db():
    """
    Consulta la base de datos 

    
    Retorna un DataFrame.
    """
    conexion = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD
    )
    
    query = query = f"""
    SELECT 
        e.idEvento, 
        e.fecha_evento, 
      
        e.coordenada_longitud, 
        e.coordenada_latitud,
        bar.id
        
    FROM 
        fullclean_contactos.vwEventos e
    LEFT JOIN 
        fullclean_contactos.vwContactos con ON e.id_contacto = con.id
    LEFT JOIN 
        fullclean_contactos.barrios bar ON bar.id = con.id_barrio
    LEFT JOIN 
        fullclean_contactos.ciudades ciu ON ciu.id = con.id_ciudad
    WHERE 
        e.fecha_evento BETWEEN '2025-06-25' AND '2025-07-10'
        AND e.id_evento_tipo = 15
        AND ciu.id_centroope = 6
        AND coordenada_longitud <> 0 
        AND coordenada_latitud <> 0;
    
    """
    df = pd.read_sql(query, conexion)
    print("CONSULTA REALIZADA")
    conexion.close()
    return df

df = consultar_db()

DB_HOST: fullcontrol-aurora57.cdvmtvv1ylzx.us-east-2.rds.amazonaws.com


C:\Users\hp\AppData\Local\Temp\ipykernel_18136\4117436383.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


CONSULTA REALIZADA


In [18]:
df.shape

(411, 5)

In [28]:
from sklearn.cluster import DBSCAN
import numpy as np
import folium
from folium import FeatureGroup
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Asegurar coordenadas como float
df['coordenada_latitud'] = df['coordenada_latitud'].astype(float)
df['coordenada_longitud'] = df['coordenada_longitud'].astype(float)

# Seleccionar top 3 barrios
top_barrios = df['id'].value_counts().head(3).index.tolist()

# Función para los 10 puntos más cercanos en el cluster más denso
def obtener_10_puntos_mas_cercanos(grupo, eps=0.001):
    grupo = grupo.copy()
    coords = grupo[['coordenada_latitud', 'coordenada_longitud']].to_numpy()
    db = DBSCAN(eps=eps, min_samples=3).fit(coords)
    grupo['cluster'] = db.labels_
    cluster_valido = grupo[grupo['cluster'] != -1]
    if cluster_valido.empty:
        return grupo.head(10)
    cluster_mas_grande = cluster_valido['cluster'].value_counts().idxmax()
    puntos_cluster = cluster_valido[cluster_valido['cluster'] == cluster_mas_grande].copy()
    if puntos_cluster.empty:
        return grupo.head(10)
    centroide = puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].mean().values
    puntos_cluster['distancia'] = np.linalg.norm(
        puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].values - centroide, axis=1
    )
    return puntos_cluster.sort_values('distancia').head(10)

# Crear mapa
centro_mapa = [df['coordenada_latitud'].median(), df['coordenada_longitud'].median()]
m = folium.Map(location=centro_mapa, zoom_start=14)

# Colores únicos
colores = list(mcolors.TABLEAU_COLORS.values())
barrio_colores = {barrio: colores[i % len(colores)] for i, barrio in enumerate(top_barrios)}

# Procesar y dibujar solo los 10 puntos más cercanos por barrio
for barrio_id in top_barrios:
    grupo_completo = df[df['id'] == barrio_id]
    grupo = obtener_10_puntos_mas_cercanos(grupo_completo).sort_values('idEvento')
    coordenadas = list(zip(grupo['coordenada_latitud'], grupo['coordenada_longitud']))
    color = barrio_colores[barrio_id]

    # Puntos
    for i, (lat, lon) in enumerate(coordenadas):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=f"Barrio ID: {barrio_id} - Orden: {i+1}"
        ).add_to(m)

    # Líneas entre puntos
    for i in range(len(coordenadas)-1):
        start = coordenadas[i]
        end = coordenadas[i+1]
        if all(np.isfinite(start)) and all(np.isfinite(end)):
            folium.PolyLine(
                locations=[start, end],
                color=color,
                weight=3,
                opacity=0.9,
                tooltip=f"{i+1} → {i+2}"
            ).add_to(m)

    # Flecha final como ícono visual
    if len(coordenadas) >= 2:
        p1 = coordenadas[-2]
        p2 = coordenadas[-1]
        dx = p2[1] - p1[1]
        dy = p2[0] - p1[0]
        angle = (np.degrees(np.arctan2(dy, dx)) + 360) % 360

        # Determinar ícono basado en dirección
        if 45 <= angle < 135:
            arrow_icon = 'arrow-up'
        elif 135 <= angle < 225:
            arrow_icon = 'arrow-left'
        elif 225 <= angle < 315:
            arrow_icon = 'arrow-down'
        else:
            arrow_icon = 'arrow-right'

        folium.Marker(
            location=p2,
            icon=folium.Icon(icon=arrow_icon, prefix='fa', color='black'),
            popup="Fin del recorrido"
        ).add_to(m)

# Mostrar mapa
m


In [13]:
print(df_top[['coordenada_latitud', 'coordenada_longitud']].dtypes)
print(df_top[['coordenada_latitud', 'coordenada_longitud']].head())


coordenada_latitud     object
coordenada_longitud    object
dtype: object
    coordenada_latitud coordenada_longitud
97   5.041008333333334          -75.509515
100  5.039491666666667  -75.50993833333334
143          5.0377998         -75.5092582
150            5.03664  -75.50945833333334
151         5.03653563        -75.50958673


In [24]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sklearn.cluster import DBSCAN
import numpy as np
# 1. Seleccionar los 3 barrios con más muestras
top_barrios = df['id'].value_counts().head(3).index.tolist()

# 2. Tomar 10 registros por barrio, ordenados por idEvento
df_top = df[df['id'].isin(top_barrios)].copy()
df_top = df_top.sort_values(['id', 'idEvento'])

def obtener_10_puntos_mas_cercanos(grupo, eps=0.001):
    # Asegurar que coordenadas estén en float
    grupo = grupo.copy()
    grupo['coordenada_latitud'] = grupo['coordenada_latitud'].astype(float)
    grupo['coordenada_longitud'] = grupo['coordenada_longitud'].astype(float)

    coords = grupo[['coordenada_latitud', 'coordenada_longitud']].to_numpy()
    db = DBSCAN(eps=eps, min_samples=3).fit(coords)
    grupo['cluster'] = db.labels_

    # Filtrar clusters válidos
    cluster_valido = grupo[grupo['cluster'] != -1]
    if cluster_valido.empty:
        return grupo.head(10)

    # Seleccionar cluster más grande
    cluster_mas_grande = cluster_valido['cluster'].value_counts().idxmax()
    puntos_cluster = cluster_valido[cluster_valido['cluster'] == cluster_mas_grande].copy()

    if puntos_cluster.empty:
        return grupo.head(10)

    # Calcular centroide y distancias
    centroide = puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].mean().values
    puntos_cluster['distancia'] = np.linalg.norm(
        puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].values - centroide, axis=1
    )

    return puntos_cluster.sort_values('distancia').head(10)


df_top =obtener_10_puntos_mas_cercanos(df_top[df_top['id'] == top_barrios[0]])



In [25]:



# 3. Crear mapa centrado en la mediana de las coordenadas
centro_mapa = [df_top['coordenada_latitud'].median(), df_top['coordenada_longitud'].median()]
m = folium.Map(location=centro_mapa, zoom_start=14)

# 4. Generar colores únicos para cada barrio
colores = list(mcolors.TABLEAU_COLORS.values())
barrio_colores = {barrio: colores[i % len(colores)] for i, barrio in enumerate(top_barrios)}

## Asegurar que las coordenadas sean float
df_top['coordenada_latitud'] = df_top['coordenada_latitud'].astype(float)
df_top['coordenada_longitud'] = df_top['coordenada_longitud'].astype(float)

# Redibujar coordenadas
for barrio_id in top_barrios:
    grupo = df_top[df_top['id'] == barrio_id].copy()
    grupo = grupo.sort_values('idEvento')
    coordenadas = list(zip(grupo['coordenada_latitud'], grupo['coordenada_longitud']))
    color = barrio_colores[barrio_id]
    
    # Puntos
    for i, (lat, lon) in enumerate(coordenadas):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=f"Barrio ID: {barrio_id} - Orden: {i+1}"
        ).add_to(m)
    
    # Segmentos con validación
    for i in range(len(coordenadas)-1):
        start = [float(coordenadas[i][0]), float(coordenadas[i][1])]
        end = [float(coordenadas[i+1][0]), float(coordenadas[i+1][1])]
        
        if all(np.isfinite(start)) and all(np.isfinite(end)):
            folium.PolyLine(
                locations=[start, end],
                color=color,
                weight=3,
                opacity=0.9,
                tooltip=f"→ {i+1} a {i+2}"
            ).add_to(m)
        else:
            print(f"❌ Coordenadas inválidas: {start} -> {end}")

# 5. Añadir texto a las líneas
# Mostrar mapa

m

In [26]:
from sklearn.cluster import DBSCAN
import numpy as np
import folium
from folium import FeatureGroup
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Convertir coordenadas a float
df['coordenada_latitud'] = df['coordenada_latitud'].astype(float)
df['coordenada_longitud'] = df['coordenada_longitud'].astype(float)

# Seleccionar los 3 barrios con más muestras
top_barrios = df['id'].value_counts().head(3).index.tolist()

# Función para obtener los 10 puntos más cercanos dentro del cluster más denso
def obtener_10_puntos_mas_cercanos(grupo, eps=0.001):
    grupo = grupo.copy()
    coords = grupo[['coordenada_latitud', 'coordenada_longitud']].to_numpy()
    db = DBSCAN(eps=eps, min_samples=3).fit(coords)
    grupo['cluster'] = db.labels_
    cluster_valido = grupo[grupo['cluster'] != -1]
    if cluster_valido.empty:
        return grupo.head(10)
    cluster_mas_grande = cluster_valido['cluster'].value_counts().idxmax()
    puntos_cluster = cluster_valido[cluster_valido['cluster'] == cluster_mas_grande].copy()
    if puntos_cluster.empty:
        return grupo.head(10)
    centroide = puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].mean().values
    puntos_cluster['distancia'] = np.linalg.norm(
        puntos_cluster[['coordenada_latitud', 'coordenada_longitud']].values - centroide, axis=1
    )
    return puntos_cluster.sort_values('distancia').head(10)

# Aplicar a los 3 barrios y concatenar
df_conectado = pd.concat([
    obtener_10_puntos_mas_cercanos(df[df['id'] == barrio])
    for barrio in top_barrios
])

# Crear el mapa
centro_mapa = [df_conectado['coordenada_latitud'].median(), df_conectado['coordenada_longitud'].median()]
m = folium.Map(location=centro_mapa, zoom_start=15)

# Colores únicos
colores = list(mcolors.TABLEAU_COLORS.values())
barrio_colores = {barrio: colores[i % len(colores)] for i, barrio in enumerate(top_barrios)}

# Dibujar por barrio
for barrio_id in top_barrios:
    grupo = df_conectado[df_conectado['id'] == barrio_id].sort_values('idEvento')
    coordenadas = list(zip(grupo['coordenada_latitud'], grupo['coordenada_longitud']))
    color = barrio_colores[barrio_id]

    # Dibujar puntos
    for i, (lat, lon) in enumerate(coordenadas):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=f"Barrio ID: {barrio_id} - Orden: {i+1}"
        ).add_to(m)

    # Dibujar segmentos
    for i in range(len(coordenadas)-1):
        start = [float(coordenadas[i][0]), float(coordenadas[i][1])]
        end = [float(coordenadas[i+1][0]), float(coordenadas[i+1][1])]
        if all(np.isfinite(start)) and all(np.isfinite(end)):
            folium.PolyLine(
                locations=[start, end],
                color=color,
                weight=3,
                opacity=0.9,
                tooltip=f"{i+1} → {i+2}"
            ).add_to(m)

    # Flecha en el último punto
    if len(coordenadas) >= 2:
        p1 = coordenadas[-2]
        p2 = coordenadas[-1]
        dx = p2[1] - p1[1]
        dy = p2[0] - p1[0]
        angle = np.degrees(np.arctan2(dy, dx))

        folium.RegularPolygonMarker(
            location=p2,
            number_of_sides=3,
            radius=10,
            rotation=angle,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup="Fin del recorrido"
        ).add_to(m)

# Mostrar mapa
m
